In [13]:
from numpy import *

In [14]:
#4.5.1 准备数据

In [15]:
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]    #1 is abusive, 0 not
    return postingList, classVec

In [16]:
##创建词表集合
def createVocabList(dataSet):
    vocabSet = set()
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [17]:
##判断某一列单词是否出现在词表中
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my vocabulary!" % word)
    return returnVec

In [18]:
listPosts,listClasses = loadDataSet()

In [19]:
myVocabList = createVocabList(listPosts)

In [20]:
#4.5.2训练算法

In [27]:
##朴素贝叶斯训练函数
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num,p1Num = ones(numWords),ones(numWords)
    p0Denom,p1Denom = 2.0,2.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])     
    p1Vect = log(p1Num/p1Denom)
    p0Vect = log(p0Num/p0Denom)    
    return p0Vect,p1Vect,pAbusive

In [28]:
trainMat = []
for postinDoc in listPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))

In [29]:
p0V,p1V,pAb = trainNB0(trainMat,listClasses)

In [30]:
pAb

0.5

In [31]:
p0V

array([-2.56494936, -3.25809654, -1.87180218, -2.56494936, -3.25809654,
       -2.56494936, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
       -3.25809654, -2.56494936, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -2.15948425, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -2.56494936, -3.25809654, -3.25809654, -3.25809654,
       -3.25809654, -2.56494936, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -3.25809654])

In [32]:
p1V

array([-3.04452244, -2.35137526, -3.04452244, -3.04452244, -2.35137526,
       -3.04452244, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
       -1.65822808, -3.04452244, -1.94591015, -2.35137526, -2.35137526,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
       -1.94591015, -3.04452244, -2.35137526, -2.35137526, -2.35137526,
       -2.35137526, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
       -2.35137526, -2.35137526])

In [ ]:
#测试算法

In [33]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #概率相乘，再取对数后，转为直接相加
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [36]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(trainMat,listClasses)
    
    testEntry = ['love','my','dalmation']
    thisDoc = setOfWords2Vec(myVocabList,testEntry)
    
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))

In [37]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0


In [38]:
#4.5.4 词袋模型

In [40]:
def bagofWords2VecMN(vocabList,inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [41]:
from pathlib import Path
data_path=Path('D:\\python_algorithm\\machinelearinginaction\\《机器学习实战》Python3代码\\Ch04')

fr = open(data_path / 'email/ham/6.txt').read()

In [43]:
import re
regEx = re.compile('\\W*')
listOfEokens = regEx.split(fr)

In [44]:
#4.6.2 测试算法

In [45]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [59]:
def spamTest():
    docList = []
    classList = []
    fullText = []
    for i in range(1,26):
        file_name = 'email/spam/%d.txt'%i
        wordList = textParse(open(data_path / file_name,encoding="ISO-8859-1").read())     #加入指定的编码格式
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        
        file_name = 'email/ham/%d.txt'%i
        wordList = textParse(open(data_path / file_name,encoding="ISO-8859-1").read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    
    vocabList = createVocabList(docList)
    trainingSet = range(50)
    testSet = []
    
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(list(trainingSet)[randIndex])                    #转为list
        
    trainMat = []
    trainClasses = []
    
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
        
    p0V,p1V,pSpam = trainNB0(trainMat,trainClasses)
    
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])
        if classifyNB(wordVector,p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is:',float(errorCount)/len(testSet))
    

In [60]:
spamTest()

the error rate is: 0.6
